In [1]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils
using LinearAlgebra

# Plots
using CairoMakie

In [11]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 1
const n_levels = 2

t_f = 20 * ns
n_steps = 101
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

0.2

In [12]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
X0=kron(Paulis["X"],Paulis["I"])
Y0=kron(Paulis["Y"],Paulis["I"])
X1=kron(Paulis["I"],Paulis["X"])
Y1=kron(Paulis["I"],Paulis["Y"])

excitation(theta) = exp(-im/2 * theta * (X0 * Y1-Y0 * X1))

excitation (generic function with 1 method)

In [13]:
a = [0 1; 0 0]
ad = transpose(a)
a0  = kron(a,Paulis["I"])
ad0  = kron(ad,Paulis["I"])
a1  = kron(Paulis["I"],a)
ad1  = kron(Paulis["I"],ad);

In [14]:
H_drift = [ ]
H_drives = [
    ad0 * a0,
    ad1 * a1,
    #a0+ad0,
    #im * (a0-ad0),
    #a1+ad1,
    #im * (a1-ad1),
    ad0 * a1 + a0 * ad1
]
system = QuantumSystem(H_drives);

In [15]:
PICO_max_iter = 250

# Shape the cost function with weights on states and controls
Q = 100.
R =1e-4
# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .02
infidelities = []
trajectories = []
for theta in range(0,2*pi,21)
    temp_trajectories =[]
    temp_infidelities=[]
    print("-----------------------")
    print("Angle "*string(theta))
    println("-----------------------")
    target = excitation(theta)
    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        verbose=true,
        hessian_approximation=true,
        pade_order=10,
        free_time=false,
        timesteps_all_equal=true,
        subspace=[1,2,3,4],
        max_iter=PICO_max_iter,
    )
    solve!(p)
    result = copy(p.trajectory)
    push!(trajectories,result)
    states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))
    println("Infidelity " *string(unitary_infidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target))))
    rollout_states = unitary_rollout(result, system; integrator=exp)
    ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
    println("Rollout Error "*string(maximum(ΔUs)))
    println("Infidelity "*string(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))))
    push!(infidelities,unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target)))
end



-----------------------Angle 0.0-----------------------
    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:    25642
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     4103
                     variables with only lower bounds:        

  76  5.2974378e-07 1.22e-08 6.82e-04  -6.1 6.27e-04    -  1.00e+00 1.00e+00H  1
  77  3.9414034e-09 2.05e-08 3.29e-04  -6.1 4.78e-04    -  1.00e+00 1.00e+00h  1
  78  4.2782867e-09 1.49e-07 1.73e+00  -4.0 2.64e-02    -  1.00e+00 4.95e-02h  5
  79  8.9461087e-07 8.05e-09 4.83e-04  -4.1 3.53e-04    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  1.5515109e-06 2.95e-12 5.20e-04  -6.1 4.63e-04    -  1.00e+00 1.00e+00H  1
  81  1.4913080e-06 3.79e-09 6.08e-05  -6.1 2.30e-04    -  1.00e+00 1.00e+00h  1
  82  8.1220946e-08 3.84e-09 5.42e-04  -6.1 2.30e-04    -  1.00e+00 1.00e+00h  1
  83  1.7613266e-07 2.89e-09 4.87e-04  -6.1 4.31e-04    -  1.00e+00 2.50e-01h  3
  84  6.7348097e-08 3.25e-09 5.47e-04  -6.1 7.65e-03    -  1.00e+00 1.56e-02h  7
  85  2.7854957e-08 2.83e-09 5.30e-04  -6.1 4.58e-04    -  1.00e+00 2.50e-01h  3
  86  2.0144063e-05 1.11e-15 1.77e-03  -6.1 5.30e-04    -  1.00e+00 1.00e+00H  1
  87  1.2173350e-05 1.37e-12

 169  5.2559783e-10 3.33e-16 3.99e-07  -9.0 3.66e-08    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  5.2559783e-10 3.33e-16 3.99e-07  -9.0 1.39e-07    -  1.00e+00 6.10e-05h 15
 171  5.2559783e-10 3.33e-16 3.99e-07  -9.0 1.39e-07    -  1.00e+00 1.91e-06h 20
 172  5.2559783e-10 3.33e-16 3.99e-07  -9.0 1.39e-07    -  1.00e+00 3.81e-06h 19
 173  5.2546422e-10 3.33e-16 3.48e-07  -9.0 1.39e-07    -  1.00e+00 1.25e-01h  4
 174  5.2543084e-10 3.33e-16 3.38e-07  -9.0 3.38e-09   2.0 1.00e+00 1.00e+00H  1
 175  5.2512860e-10 1.11e-16 1.44e-09  -9.0 1.17e-07    -  1.00e+00 1.00e+00H  1
 176  5.2512851e-10 2.22e-16 1.44e-09  -9.0 5.07e-10    -  1.00e+00 1.00e+00h  1
 177  5.2500912e-10 2.90e-14 2.34e-09  -9.0 6.13e-03    -  1.00e+00 1.22e-04h 14
 178  1.2016845e-03 1.48e-15 1.64e-02  -9.0 6.58e-03    -  1.00e+00 1.00e+00H  1
 179  1.2016835e-03 1.32e-14 1.64e-02  -9.0 8.02e-01    -  2.95e-02 4.86e-07f 17
iter    objective    inf_pr 

            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:    25642
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     4103
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      600
                     variables with only upper bounds:        0
Total number of equality constraints.................:     3800
Total number of inequality constraints...............:

  79  4.1804351e-05 2.08e-03 2.52e+01  -4.0 2.44e-01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  4.8640970e-03 1.73e-03 2.49e+01  -4.1 2.32e-01    -  1.00e+00 1.00e+00h  1
  81  1.3214629e-03 3.21e-03 2.50e+01  -4.1 3.37e-01    -  1.00e+00 1.00e+00h  1
  82  2.8143033e-03 3.09e-03 2.50e+01  -4.1 2.74e+00    -  9.46e-01 2.73e-02h  5
  83  4.5611980e-04 8.22e-04 2.50e+01  -4.1 1.64e-01    -  1.00e+00 1.00e+00h  1
  84  1.2769800e-03 7.06e-04 2.50e+01  -4.1 3.42e-01    -  1.00e+00 2.50e-01h  3
  85  3.5184694e-04 3.21e-04 2.49e+01  -4.1 1.25e-01    -  1.00e+00 1.00e+00h  1
  86  1.8921171e-03 4.41e-04 2.51e+01  -4.1 7.23e-01    -  1.00e+00 1.25e-01h  4
  87  8.8861886e-04 1.72e-04 2.52e+01  -4.1 8.41e-02    -  1.00e+00 1.00e+00h  1
  88  6.9588276e-04 2.43e-04 2.49e+01  -4.1 6.46e-01    -  1.00e+00 1.25e-01h  4
  89  3.8342357e-04 9.49e-05 2.51e+01  -4.1 5.54e-02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr 

...

   1  4.2672395e+00 8.58e-01 5.00e+01  -1.1 6.81e-01    -  2.10e-02 2.49e-01f  1
   2  3.9784315e+00 8.04e-01 4.87e+01  -0.1 9.48e-01    -  7.72e-01 6.28e-02f  1
   3  1.9399645e+00 2.36e-01 2.56e+01   0.2 4.58e-01    -  6.95e-01 7.07e-01f  1
   4  3.2995705e+00 6.30e-03 2.99e+01  -1.4 1.71e-01    -  7.93e-01 1.00e+00h  1
   5  2.1050843e+00 5.07e-04 6.56e+00  -1.3 1.23e-01    -  9.97e-01 1.00e+00f  1
   6  9.3047340e-01 7.98e-04 1.55e+00  -2.0 1.32e-01    -  9.98e-01 1.00e+00f  1
   7  9.7543797e-01 2.46e-04 5.11e-01  -2.8 5.53e-02    -  1.00e+00 1.00e+00h  1
   8  2.8477344e-01 1.21e-03 4.86e+01  -2.4 1.05e+00    -  1.00e+00 1.46e-01f  2
   9  7.4597342e-01 3.58e-04 2.03e+01  -2.5 1.01e-01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.8571514e-01 1.77e-04 1.53e+01  -3.2 5.60e-02    -  1.00e+00 1.00e+00f  1
  11  1.5319114e-01 4.45e-04 3.85e+01  -2.9 1.93e-01    -  1.00e+00 3.92e-01f  1
  12  1.2516495e-01 1.91e-05

  94  1.0438035e-02 3.24e-05 3.31e+01  -4.1 3.67e-02    -  1.00e+00 2.50e-01h  3
  95  2.7420391e-03 1.34e-05 2.58e+01  -4.1 1.30e-02    -  1.00e+00 1.00e+00h  1
  96  2.1232563e-06 1.18e-05 2.68e+01  -4.0 1.86e-02    -  1.00e+00 2.50e-01h  3
  97  1.7130326e-03 1.17e-05 2.25e+01  -4.0 3.48e-02    -  1.00e+00 1.04e-01h  4
  98  1.7833475e-02 2.51e-07 5.00e+01  -4.1 2.15e-02    -  1.00e+00 1.00e+00H  1
  99  9.6803181e-03 3.35e-06 3.61e+01  -4.1 5.95e-03    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  3.1209018e-03 5.58e-06 4.19e+01  -4.1 3.29e-02    -  1.00e+00 2.50e-01f  3
 101  1.4927866e-03 8.47e-06 1.95e+01  -4.0 8.97e-02    -  1.00e+00 8.34e-02h  4
 102  1.1909019e-02 2.58e-07 5.00e+01  -4.0 2.17e-02    -  1.00e+00 1.00e+00H  1
 103  7.3877334e-03 2.89e-06 3.39e+01  -4.1 6.74e-03    -  1.00e+00 1.00e+00f  1
 104  1.9263198e-04 1.08e-05 4.40e+01  -4.1 4.82e-02    -  1.00e+00 2.50e-01f  3
 105  4.5798744e-03 8.95e-07

 187  1.8787561e-06 2.65e-09 2.63e-04  -4.0 4.03e-04    -  1.00e+00 1.00e+00H  1
 188  2.5325997e-06 4.08e-09 4.40e+01  -4.0 5.74e-04    -  1.00e+00 5.00e-01h  2
 189  2.1185096e-06 6.24e-08 6.23e+01  -4.1 1.14e-03    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190  3.3348937e-06 4.50e-08 2.33e-01  -4.1 1.43e-03    -  1.00e+00 5.00e-01h  2
 191  7.0107158e-06 2.99e-07 1.62e+01  -4.1 2.36e-03    -  1.00e+00 1.00e+00h  1
 192  2.0441824e-06 2.12e-07 1.66e+01  -4.1 3.09e-03    -  1.00e+00 5.00e-01h  2
 193  4.7251013e-06 1.60e-07 3.20e+01  -4.1 2.70e-03    -  1.00e+00 2.50e-01h  3
 194  2.3724989e-06 1.49e-07 3.05e+01  -4.1 1.79e-03    -  1.00e+00 1.00e+00h  1
 195  4.5319439e-06 1.41e-07 1.87e+01  -4.1 3.40e-03    -  1.00e+00 1.25e-01h  4
 196  1.9335678e-06 2.38e-08 3.13e+01  -4.1 6.76e-04    -  1.00e+00 1.00e+00h  1
 197  4.5671516e-06 1.88e-07 3.39e+01  -4.1 3.84e-03    -  1.00e+00 5.00e-01h  2
 198  4.4806720e-06 2.71e-08

   1  3.3231055e+00 8.42e-01 5.00e+01  -1.2 6.42e-01    -  1.72e-02 2.70e-01f  1
   2  3.1196207e+00 7.91e-01 4.91e+01  -0.1 9.43e-01    -  8.65e-01 6.09e-02f  1
   3  2.9360246e-02 2.59e-02 2.43e+01   0.1 4.56e-01    -  7.37e-01 9.67e-01f  1
   4  8.9480900e-01 9.14e-04 3.56e+01  -1.6 1.94e-01    -  9.61e-01 1.00e+00h  1
   5  9.3074273e-02 2.28e-04 4.52e+01  -2.1 7.40e-02    -  1.00e+00 1.00e+00f  1
   6  5.3372246e-02 1.31e-04 2.73e+01  -2.8 4.63e-02    -  1.00e+00 5.00e-01f  2
   7  1.9103800e-02 5.36e-05 2.94e+01  -3.4 3.77e-02    -  1.00e+00 1.00e+00h  1
   8  1.9767174e-02 2.47e-06 2.35e+01  -4.0 8.52e-03    -  1.00e+00 1.00e+00h  1
   9  5.7968625e-03 5.98e-06 1.91e+01  -4.0 1.13e-02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  5.1453821e-03 9.19e-06 3.20e+01  -3.6 1.52e-01    -  1.00e+00 5.66e-02f  4
  11  5.5240199e-04 1.43e-06 2.71e+01  -4.0 6.58e-03    -  1.00e+00 1.00e+00h  1
  12  5.1643904e-04 1.78e-06

  94  2.8303138e-08 2.07e-08 2.50e+01  -4.1 8.49e-04    -  1.00e+00 1.00e+00h  1
  95  8.1090544e-09 2.98e-08 2.50e+01  -4.1 9.72e-04    -  1.00e+00 1.00e+00h  1
  96  7.5049671e-09 4.77e-09 1.25e-03  -4.1 5.68e-04    -  1.00e+00 1.00e+00h  1
  97  2.1587820e-09 3.38e-09 1.24e-03  -4.0 6.52e-04    -  1.00e+00 5.00e-01h  2
  98  2.5820394e-10 5.45e-09 1.24e-03  -4.1 3.51e-03    -  1.00e+00 1.25e-01h  4
  99  6.1384855e-08 5.47e-09 1.17e-03  -4.0 1.71e-03    -  1.00e+00 1.25e-01h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  1.4804660e-07 5.31e-09 1.13e-03  -4.0 1.61e-03    -  1.00e+00 6.25e-02h  5
 101  2.7511969e-07 5.16e-09 1.09e-03  -4.0 1.57e-03    -  1.00e+00 6.25e-02h  5
 102  3.8790730e-05 2.85e-11 2.62e-03  -4.0 1.04e-03    -  1.00e+00 1.00e+00H  1
 103  2.8181724e-05 3.30e-09 2.18e-03  -4.0 2.21e-04    -  1.00e+00 1.00e+00h  1
 104  1.5228632e-05 5.70e-08 2.52e-04  -4.0 1.44e-03    -  1.00e+00 1.00e+00H  1
 105  3.9049864e-07 9.31e-10

In [16]:
infidelities

21-element Vector{Any}:
 6.644373673481141e-8
 4.353124396727637e-5
 2.054651551652853e-5
 0.000250315610139884
 7.997174431562115e-7
 3.750408800629046e-6
 8.701507920871876e-7
 0.0016369509724326603
 4.09151845737199e-9
 0.000256227870008674
 1.383337888682945e-13
 7.549545118057388e-6
 1.5492661221694348e-6
 2.9734190665209326e-9
 0.0019230925044361191
 4.1182918488980036e-5
 4.553376025007161e-6
 1.9007529790004973e-5
 1.2806131377551822e-8
 1.542906336027272e-8
 6.616929226765933e-14